In [6]:
import netCDF4 as nc
from netCDF4 import Dataset
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import cmocean as cmo
import os,sys,fnmatch,time
import numpy as np
import matplotlib.gridspec as gridspec
from matplotlib import animation
from salishsea_tools.nc_tools import scDataset
from matplotlib import colors
from matplotlib import ticker
from matplotlib.colors import LinearSegmentedColormap
from mpl_toolkits.axes_grid1.inset_locator import InsetPosition
import matplotlib as mpl
import numpy.ma as ma
from scipy import interpolate
import matplotlib.gridspec as gridspec
import matplotlib.patches as patches

%matplotlib inline

import sys
sys.path.append('/ocean/imachuca/Canyons/analysis_mackenzie_canyon/notebooks/upwelling_depth/')
import upwelling_functions
sys.path.append('/ocean/imachuca/Canyons/analysis_mackenzie_canyon/notebooks/general_circulation/')
import general_functions
import quicklook

import warnings
warnings.filterwarnings("ignore")

import time
from datetime import datetime 

In [7]:
def get_varb_surf(vosaline_array, salt, varb):
    
    if len(vosaline_array.shape)==1:
        f = interpolate.interp1d(vosaline_array, varb)
        ynew = f(salt)
        varb_array = ynew
        
    elif len(vosaline_array.shape)!=1:
        varb_array = np.full([vosaline_array.shape[-2], vosaline_array.shape[-1]], np.nan)
        for i in range(vosaline_array.shape[-1]):
            for j in range(vosaline_array.shape[-2]):
                vosaline_column = vosaline_array[:, j, i]
                if len(varb.shape)==1:
                    varb_column = varb
                elif len(varb.shape)==3:
                    varb_column = varb[:, j, i]

                try:
                    f = interpolate.interp1d(vosaline_column, varb_column)
                    ynew = f(salt)
                    varb_array[j, i] = ynew
                except ValueError:
                    varb_array[j, i] = np.nan  
                
    return varb_array

In [8]:
def get_varbs(kind, case, fname, meshmaskname, time_s):
    
    dirname = '/ocean/imachuca/Canyons/results_mackenzie/extended_domain/'+kind+'_'+case+'/'
    
    filesW = general_functions.get_files(dirname, fname, 'grid_W')
    filesT = general_functions.get_files(dirname, fname, 'grid_T')
    
    y,x = slice(1,-1,None), slice(1,-1,None)
    
    with scDataset(filesW) as dsW, scDataset(filesT) as dsT:
        vovecrtz = dsW.variables['vovecrtz'][time_s, :, y, x]
        vosaline = dsT.variables['vosaline'][time_s, :, y, x]
        
    with nc.Dataset(os.path.join(dirname, meshmaskname), 'r') as dsM:
        e1t0 = np.nanmean(dsM.variables['e1t'][0, y, x], axis=0)
        e2t0 = np.nanmean(dsM.variables['e2t'][0, y, x], axis=1)
        e1t = np.cumsum(e1t0) / 1000 # km
        e2t = np.cumsum(e2t0) / 1000 # km
        
    #print(fname)
        
    return vovecrtz, vosaline, e1t, e2t

In [9]:
def get_varbs_series(kind, case, fname, meshmaskname, time_all):
    vovecrtz_c = np.full([len(time_all), 80, 242, 272], np.nan)
    vosaline_c = np.full([len(time_all), 80, 242, 272], np.nan)
    for time_s, n in zip(time_all, np.arange(len(time_all))):
        vovecrtz, vosaline, e1t_c, e2t_c = get_varbs(kind, case, fname, meshmaskname, time_s)
        vovecrtz_c[n, :, :, :] = vovecrtz
        vosaline_c[n, :, :, :] = vosaline
    return vovecrtz_c, vosaline_c, e1t_c, e2t_c

In [10]:
def plot_map(ax, surface_time_c, t_ind, time_all, x_shift, y_shift, mid, labs, vm):
    
    #print(time_all, t_ind, time_all[t_ind])
    
    fs_numb, fs_str = 25, 30
        
    cmap = cmo.cm.balance
    #cmap.set_bad('#A69F98')
    cmap.set_bad('#8b7765')
       
    p = ax.pcolormesh(x_shift, y_shift, surface_time_c[t_ind, :, :], vmin=-vm, vmax=vm, cmap=cmo.cm.balance)
    
    ax.tick_params(axis='both', which='both', length=0, labelsize=fs_numb)
    ax.set_aspect(aspect='equal')
    ax.set_xlabel('Alongshore (x)\ndistance [km]', fontsize=fs_str)

    min_x, max_x, min_y, max_y = mid-100, mid+100, 25, 200
    ax.set_xlim([min_x, max_x])
    ax.set_ylim([min_y, max_y])
    
#     ind_y = 45
#     ind_y_mid = ind_y+(ind_y-min_x)*0.5
#     ind_x_mid = min_x+(ind_y-min_x)*0.2
    ax.add_patch(patches.Rectangle((min_x, max_y-26),(25),(25),
                                   fc='w', ec='k', lw=1.5, alpha=1, zorder=8))
    ax.annotate(labs, xy=(min_x+10, max_y-15),
                fontsize=fs_str, weight='bold', va='center', ha='center', zorder=9)
    
    ax.annotate('hour '+str(time_all[t_ind]), xy=(max_x-80, 35), fontsize=fs_numb)
        
    return ax, p

In [11]:
def plot_streamlines(ax, kind, case, fname, meshmaskname, t_ind, time_all, vosaline_c, salt, x_shift, y_shift):
    dirname = '/ocean/imachuca/Canyons/results_mackenzie/extended_domain/'+kind+'_'+case+'/'
    
    filesU = general_functions.get_files(dirname, fname, 'grid_U')        
    filesV = general_functions.get_files(dirname, fname, 'grid_V')
    
    y,x = slice(1,-1,None), slice(1,-1,None)
    
    time_s = time_all[t_ind]
    
    with scDataset(filesU) as dsU, scDataset(filesV) as dsV:
        vozocrtx = dsU.variables['vozocrtx'][time_s, :, y, x]
        vomecrty = dsV.variables['vomecrty'][time_s, :, y, x]
    
    u_daily = get_varb_surf(vosaline_c[t_ind, :, :, :], salt, vozocrtx)
    v_daily = get_varb_surf(vosaline_c[t_ind, :, :, :], salt, vomecrty)
    x_slice_c, y_slice_c, u_nstg, v_nstg, speeds = quicklook.get_speeds(u_daily, v_daily, 1)
    
    x_points_c, y_points_c = x_shift[x_slice_c], y_shift[y_slice_c]
    strm = ax.streamplot(x_points_c, y_points_c, u_nstg, v_nstg, color='k', density=2, linewidth=1)
        
    return ax

In [12]:
def plot_hovmoller(ax, vovecrtz_c, vosaline_c, e1t, e2t, salt, time_all, y_ind, x_shift, mid, vm, labs):
    #print(time_all)
    
    fs_numb, fs_str = 25, 30
    
    surface_time_c = np.full([len(time_all), vosaline_c.shape[-2], vosaline_c.shape[-1]], np.nan)

    for t in range(len(time_all)):
        varb_array = get_varb_surf(vosaline_c[t, :, :, :], salt, vovecrtz_c[t, :, :, :]) * 1000
        surface_time_c[t, :, :] = varb_array
        
    hovmoller_c = surface_time_c[:, y_ind, :]
    
    cmap = cmo.cm.balance
    #cmap.set_bad('#A69F98')
    cmap.set_bad('#8b7765')
    
    ys = np.arange(len(time_all)+1)
    
    ax.pcolormesh(x_shift, ys, hovmoller_c, vmin=-vm, vmax=vm, cmap=cmap)
    
    ax.tick_params(axis='both', which='both', length=0, labelsize=fs_numb)
    
    #print(np.nanmin(hovmoller_c), np.nanmax(hovmoller_c))
    
    min_x, max_x = mid-100, mid+100
    ax.set_xlim([min_x, max_x])
    
    ax.set_yticks(np.arange(ys[0]+0.5, ys[-1]+0.5, 1))
    #ax.yaxis.set_major_locator(plt.MaxNLocator(len(time_all)))
    ax.set_yticklabels(time_all)
    #ax.yaxis.set(ticks=ys, ticklabels=np.array([0.5, 1.5]))
    
    ax.add_patch(patches.Rectangle((min_x, ys[-2]),(25),(1),
                                   fc='w', ec='k', lw=1.5, alpha=1, zorder=8))
    ax.annotate(labs, xy=(min_x+10, ys[-2]+0.75),
                fontsize=fs_str, weight='bold', va='center', ha='center', zorder=9)

    return ax, surface_time_c

In [ ]:
time_all = np.arange(12, 73, 3) #np.array([36, 48, 60])#
salt_UHW_SA, salt_ATW_SA = 33.26258156425155, 34.99102991139694

case='base'
fname = '1_MCKNZ*'
meshmaskname = '1_mesh_mask.nc'

vovecrtz_ci, vosaline_ci, e1t_ci, e2t_ci = get_varbs_series('ideal', case, fname, meshmaskname, time_all)
vovecrtz_cr, vosaline_cr, e1t_cr, e2t_cr = get_varbs_series('real', case, fname, meshmaskname, time_all)

vovecrtz_pi, vosaline_pi, e1t_p, e2t_p = get_varbs('ideal', case, 'MCKNZ*', 'mesh_mask.nc', 0)

In [ ]:
fs_numb, fs_str = 25, 30
kinds = ['ideal', 'ideal', 'real', 'real']
salts = [salt_UHW_SA, salt_ATW_SA, salt_UHW_SA, salt_ATW_SA]
t_bots = [36,48,36,48]
labs_top = ['a', 'b', 'c', 'd']
labs_mid = ['e', 'f', 'g', 'h']
labs_bot = ['i', 'j', 'k', 'l']
y_ind_tops, y_ind_mids = [175, 175, 175, 175], [100, 159, 100, 159]

fig = plt.figure(figsize=(20, 21))
gs = gridspec.GridSpec(4, 4, width_ratios=[1,1,1,1], height_ratios=[1,1,0.2,1])

ax1 = plt.subplot(gs[0, 0])
ax2 = plt.subplot(gs[0, 1])
ax3 = plt.subplot(gs[0, 2])
ax4 = plt.subplot(gs[0, 3])

ax1b = plt.subplot(gs[1, 0])
ax2b = plt.subplot(gs[1, 1])
ax3b = plt.subplot(gs[1, 2])
ax4b = plt.subplot(gs[1, 3])

axoff = plt.subplot(gs[2,:]); axoff.axis('off')

ax5 = plt.subplot(gs[3, 0])
ax6 = plt.subplot(gs[3, 1])
ax7 = plt.subplot(gs[3, 2])
ax8 = plt.subplot(gs[3, 3])

axes_top = [ax1, ax2, ax3, ax4]
axes_mid = [ax1b, ax2b, ax3b, ax4b]
axes_bot = [ax5, ax6, ax7, ax8]

ttls = ['idealized - UHW', 'idealized - AW', 'realistic - UHW', 'realistic - AW']
for ax, ttl in zip (axes_top, ttls):
    ax.set_title(ttl, fontsize=fs_str, y=1.08)

for ax in ([ax2, ax3, ax4, ax2b, ax3b, ax4b, ax6, ax7, ax8]):
    ax.yaxis.set_tick_params(labelbottom=False)
    
for ax in (axes_top):
    ax.xaxis.set_tick_params(labelbottom=False)

for ax in ([ax1, ax2, ax3, ax4, ax1b, ax2b, ax3b, ax4b, ax5, ax6, ax7, ax8]):    
    for axis in ['top','bottom','left','right']:
        ax.spines[axis].set_linewidth(2)

vm = 1.2

for ax_top, ax_mid, ax_bot, kind, salt, y_ind_top, y_ind_mid, t_bot, lab_bot, n in zip(axes_top, axes_mid, axes_bot, kinds, salts, y_ind_tops, y_ind_mids, t_bots, labs_bot, np.arange(4)): 
    if kind == 'ideal':
        vovecrtz_c, vosaline_c, e1t_c, e2t_c = vovecrtz_ci, vosaline_ci, e1t_ci, e2t_ci
        #x_left, y_bottom = 205.822543591543,24.85159075988878
        inds = [90, 10]
        mid = 306
        ticklabs = np.arange(250,450,50)
    elif kind == 'real':
        vovecrtz_c, vosaline_c, e1t_c, e2t_c = vovecrtz_cr, vosaline_cr, e1t_cr, e2t_cr
        #x_left, y_bottom = 296.29431501679437, 24.85159075988878
        inds = [130, 10]
        mid = 400
        ticklabs = np.arange(350,550,50)
        
    
    #print('vosaline shape=',vosaline_c.shape)
    
    x_left, y_bottom = e1t_p[inds[0]], e2t_p[inds[1]]
    x_shift, y_shift = x_left+e1t_c, y_bottom+e2t_c
    
    ax_top, surface_time_c = plot_hovmoller(ax_top, vovecrtz_c, vosaline_c, e1t_c, e2t_c, 
                                            salt, time_all, y_ind_top, x_shift, mid, vm, labs_top[n])
    
    ax_mid, surface_time_c = plot_hovmoller(ax_mid, vovecrtz_c, vosaline_c, e1t_c, e2t_c, 
                                            salt, time_all, y_ind_mid, x_shift, mid, vm, labs_mid[n])
    ax_mid.set_xlabel('Alongshore (x)\ndistance [km]', fontsize=fs_str)
    
    t_ind = np.where(time_all==t_bot)[0][0]
    print('t_ind=',t_ind)
    
    ax_bot, p = plot_map(ax_bot, surface_time_c, t_ind, time_all, x_shift, y_shift, mid, lab_bot, vm)
    y_sectionA = y_bottom+e2t_c[y_ind_top]
    y_sectionB = y_bottom+e2t_c[y_ind_mid]
    print('Section A y=', y_sectionA, ', Section B y=', y_sectionB)
    ax_bot.axhline(y_sectionA, c='k', ls='--', lw=3)
    ax_bot.axhline(y_sectionB, c='k', ls='--', lw=3)
    #trans = ax_bot.get_xaxis_transform()
    print(mid+60, y_sectionA+5)
    ax_bot.annotate('A', xy=(mid+60, y_sectionA+5), fontsize=fs_numb)#, xycoords=trans)
    ax_bot.annotate('B', xy=(mid+60, y_sectionB-15), fontsize=fs_numb)#, xycoords=trans)
    
    for ax in ([ax_top, ax_mid, ax_bot]):
        ax.xaxis.set_ticks(ticklabs)
    
    ax_bot = plot_streamlines(ax_bot, kind, case, fname, meshmaskname, t_ind, time_all, vosaline_c, salt, x_shift, y_shift)

    if n==3:
        cbaxes = fig.add_axes([0.32, 0.31, 0.42, 0.02])
        cbar = fig.colorbar(p, format='%1.1f', ax=ax, cax=cbaxes, orientation='horizontal')#, ticks=ticks)
        cbar.ax.tick_params(labelsize=fs_numb)
        cbar.ax.set_xlabel('Vertical velocity [mms$^{-1}$]', fontsize=fs_str)
        

plt.text(-0.0001, 0.86, 'Section A', fontsize=fs_str, transform=plt.gcf().transFigure, rotation='vertical')
plt.text(-0.0001, 0.56, 'Section B', fontsize=fs_str, transform=plt.gcf().transFigure, rotation='vertical')
        
axes_top[0].set_ylabel('Time [hours]', fontsize=fs_str)
axes_mid[0].set_ylabel('Time [hours]', fontsize=fs_str)
axes_bot[0].set_ylabel('Cross-shore (y)\ndistance [km]', fontsize=fs_str)

fig.tight_layout()
fig.patch.set_facecolor('white')    
plt.subplots_adjust(top=0.99, wspace=0.2, hspace=0.1)
fig.savefig('./fig_hovmoller.png', dpi=200, bbox_inches='tight', format='png', facecolor=fig.get_facecolor())

t_ind= 8
Section A y= 157.39308625850535 , Section B y= 100.91231364900615
366 162.39308625850535
t_ind= 12
Section A y= 157.39308625850535 , Section B y= 145.34385476449617
366 162.39308625850535
t_ind= 8
Section A y= 157.393085820881 , Section B y= 100.91231340988548
460 162.393085820881
